<a href="https://colab.research.google.com/github/LC1332/Needy-Haruhi/blob/main/notebook/Needy%E5%90%8E%E5%8F%B0%E7%89%88%E6%9C%AC%E6%A1%86%E6%9E%B6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agent系统

我希望实现一个Agent类

这个agent有多个属性（目前设计有 Stress , Darkness和Affection）

可以通过类似agent["Stress"]这样的形式调用

请用self.attributes 字典形式存储，并且重载[]操作符使得agent的行为和字典一致

同时实现一个成员函数apply_attribute_change( attribute_change )

attribute_change是一个形如{"Darkness":-1, "Stress":1}的字典，如果字典key的值在self.attributes中存在，则累加在上面，不然则汇报warning并跳过

In [ ]:
class Agent:
    def __init__(self):
        self.attributes = {
            "Stress": 0,
            "Darkness": 0,
            "Affection": 0,
        }

    def __getitem__(self, key):
        return self.attributes.get(key)

    def __setitem__(self, key, value):
        self.attributes[key] = value

    def apply_attribute_change(self, attribute_change):
        for key, value in attribute_change.items():
            if key in self.attributes:
                self.attributes[key] += value
            else:
                print(f"Warning: {key} not in attributes, skipping")

# 示例用法
agent = Agent()
print(agent["Stress"])  # 输出 0
agent["Stress"] += 1
print(agent["Stress"])  # 输出 1
agent.apply_attribute_change({"Darkness": -1, "Stress": 1})
print(agent["Darkness"])  # 输出 -1
print(agent["Stress"])  # 输出 2
agent.apply_attribute_change({"Nonexistent": 5})  # 输出 Warning: Nonexistent not in attributes, skipping


0
1
-1
2


## Attribute String解析

我希望实现一个字符串解析函数，输入是一个string，输出是一个dict，如果字符串中出现
"Strees:", "Affection:"或者"Darkness:"，则把后面的一个有正负的浮点数作为value，对应的字符串作为key，记录在dict中

如果后面是？或者非数字，则记录成0

example input:
Stress: -1.0, Affection: +0.5
example output:
{"Stress":-1,"Affection":0.5 }

exmple input:
Affection: +4.0, Stress: -2.0, Darkness: -1.0
example output:
{"Stress":-1,"Affection":0.5 }

example input:
Affection: +2.0, Stress: -1.0, Darkness: ?
example output:
{"Affection": 2, "Stress": -1, "Darkness": 0 }

example input:
Stress: -1.0
example output:
{"Stress":-1}


In [38]:
import re

def parse_attribute_string(attribute_str):
    result = {}
    patterns = {
        "Stress": r"Stress:\s*([+-]?\d+(\.\d+)?)?",
        "Affection": r"Affection:\s*([+-]?\d+(\.\d+)?)?",
        "Darkness": r"Darkness:\s*([+-]?\d+(\.\d+)?)?"
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, attribute_str)
        if match:
            value = match.group(1)
            if value is None:
                result[key] = 0
            else:
                result[key] = float(value)

    return result

# Test cases
print(parse_attribute_string("Stress: -1.0, Affection: +0.5"))  # Output: {'Stress': -1.0, 'Affection': 0.5}
print(parse_attribute_string("Affection: +4.0, Stress: -2.0, Darkness: -1.0"))  # Output: {'Affection': 4.0, 'Stress': -2.0, 'Darkness': -1.0}
print(parse_attribute_string("Affection: +2.0, Stress: -1.0, Darkness: ?"))  # Output: {'Affection': 2.0, 'Stress': -1.0, 'Darkness': 0}
print(parse_attribute_string("Stress: -1.0"))  # Output: {'Stress': -1.0}


{'Stress': -1.0, 'Affection': 0.5}
{'Stress': -2.0, 'Affection': 4.0, 'Darkness': -1.0}
{'Stress': -1.0, 'Affection': 2.0, 'Darkness': 0}
{'Stress': -1.0}


# DialogueEvent

In [37]:
import json

class DialogueEvent:
    def __init__(self, json_str=None, user_role = None):
        if json_str:
            try:
                self.data = json.loads(json_str)
            except json.JSONDecodeError:
                print("输入的字符串不是有效的JSON格式。")
                self.data = {}
        else:
            self.data = {}

        if "Condition" not in self.data:
            self.data["Condition"] = ""

        if user_role is None:
            self.user_role = "男主"


    def __getitem__(self, key):
        return self.data.get(key, None)

    def __setitem__(self, key, value):
        self.data[key] = value

    def __repr__(self):
        return str(self.data)

    def transfer_output( self, choice_id ):
        ans = self.data["prefix"] + "\n"
        user_text = self.user_role + ": " + self.data["options"][choice_id]["user"] + "\n"
        ans += user_text
        ans += self.data["options"][choice_id]["reply"] + "\n"

        # print(self.data["options"][choice_id]['attribute_change'])
        return ans


example_json_str = """{"prefix": "糖糖: 嘿嘿，最近我在想要不要改变直播风格，你觉得我应该怎么做呀？", "options": [{"user": "你可以试试唱歌直播呀！", "reply": "糖糖: 哇！唱歌直播是个好主意！我可以把我的可爱音色展现给大家听听！谢谢你的建议！", "attribute_change": "Stress: -1.0"}, {"user": "你可以尝试做一些搞笑的小品，逗大家开心。", "reply": "糖糖: 哈哈哈，小品确实挺有趣的！我可以挑战一些搞笑角色，给大家带来欢乐！谢谢你的建议！", "attribute_change": "Stress: -1.0"}, {"user": "你可以尝试做游戏直播，和观众一起玩游戏。", "reply": "糖糖: 游戏直播也不错！我可以和观众一起玩游戏，互动更加有趣！谢谢你的建议！", "attribute_change": "Stress: -1.0"}]}"""

# 通过给定的json字符串初始化DialogueEvent实例
event = DialogueEvent(example_json_str)

# 通过类似字典的方式访问数据
# print(event["options"])  # 打印options字段的内容

print(event.transfer_output(0) )

Stress: -1.0
糖糖: 嘿嘿，最近我在想要不要改变直播风格，你觉得我应该怎么做呀？
男主: 你可以试试唱歌直播呀！
糖糖: 哇！唱歌直播是个好主意！我可以把我的可爱音色展现给大家听听！谢谢你的建议！



给定 example_json_str = """{"prefix": "糖糖: 嘿嘿，最近我在想要不要改变直播风格，你觉得我应该怎么做呀？", "options": [{"user": "你可以试试唱歌直播呀！", "reply": "糖糖: 哇！唱歌直播是个好主意！我可以把我的可爱音色展现给大家听听！谢谢你的建议！", "attribute_change": "Stress: -1.0"}, {"user": "你可以尝试做一些搞笑的小品，逗大家开心。", "reply": "糖糖: 哈哈哈，小品确实挺有趣的！我可以挑战一些搞笑角色，给大家带来欢乐！谢谢你的建议！", "attribute_change": "Stress: -1.0"}, {"user": "你可以尝试做游戏直播，和观众一起玩游戏。", "reply": "糖糖: 游戏直播也不错！我可以和观众一起玩游戏，互动更加有趣！谢谢你的建议！", "attribute_change": "Stress: -1.0"}]}"""

我希望建立一个DialogueEvent类

这个类可以凭空初始化

也可以通过DialogueEvent(str)的方式来初始化

并且json_str解析后，会以self.data的字典形式存储在类中

并且可以通过类似 event["options"]的方式进行调用

请用python为我实现

# Memory

memory我们希望Event和Memory是分离的Event的标准字段如下

- Name， Event的Name，用来后续如果玩家进行游戏修改的话可以根据
- Text， 这个event下完整的对话文本
- Embedding， text的embedding
- Condition， 这个event对应的出现条件
- Emoji， 这个memory的缩写显示emoji

Memory应该可以从Event去默认load一个

In [23]:
example_memory_json = {
    "Name": "EventName",
    "Text": "Sample Text",
    "Embedding": [0,0,0],
    "Condition": "",
    "Emoji": "😓🤯"
}

Memory会包含下面几个字段

example_memory_json = {
    "Name": "EventName",
    "Text": "Sample Text",
    "Embedding": [0,0,0],
    "Condition": "",
    "Emoji": "😓🤯"
}

请为我创建一个Memory类

这个memory类可以通过Memory(json_str)来载入

同时这个类也有和DIalogueEvent类似的get和setitem的功能

In [24]:
import json

class Memory:
    def __init__(self, json_str=None):
        if json_str:
            try:
                self.data = json.loads(json_str)
            except json.JSONDecodeError:
                print("输入的字符串不是有效的JSON格式。")
                self.data = {}
        else:
            self.data = {}

    def load_from_event( event ):
        pass

    def __getitem__(self, key):
        return self.data.get(key, None)

    def __setitem__(self, key, value):
        self.data[key] = value

    def __repr__(self):
        return str(self.data)


example_memory_json = {
    "Name": "EventName",
    "Text": "Sample Text",
    "Embedding": [0, 0, 0],
    "Condition": "",
    "Emoji": "😓🤯"
}

# 通过给定的json字符串初始化Memory实例
memory = Memory(json.dumps(example_memory_json))

# 通过类似字典的方式访问数据
print(memory["Name"])  # 打印Name字段的内容
print(memory["Emoji"])  # 打印Emoji字段的内容


EventName
😓🤯


我希望使用python实现一个简单的文字对话游戏

我希望先实现一个GameMaster类

这个类会不断的和用户对话

GameMaster类会有三个状态，

在Menu状态下，GameMaster会询问玩家是

```
1. 随机一个事件
2. 自由聊天
```

当玩家选择1的时候，GameMaster的交互会交给 EventMaster

当玩家选择2的时候，GameMaster的交互会交给 ChatMaster

当玩家在EventMaster的时候，会经历一次选择，之后就会退出

在ChatMaster的时候，如果玩家输入quit，则会退出，不然则会继续聊天。

请为我编写合适的框架，如果有一些具体的函数，可以先用pass实现。

In [ ]:
class EventMaster:
    def run(self, agent ):
        print("事件发生了！")
        # 这里可以添加事件相关的逻辑
        input("按任意键继续...")


ChatMaster实际上需要

根据agent的属性 先去filter一遍事件

然后从剩余事件中，找到和当前text最接近的k个embedding，放入ChatHaruhi架构中

In [ ]:
class MemoryPool:
    def __init__(self):
        memory = {}

    def change_memory( memory_name , new_text ):
        pass

    def retrieve( agent, query_text ):
        pass


class ChatMaster:

    def __init__(self):
        self.top_K = 7

        self.memory_pool = MemoryPool()


    def run(self, agent):
        while True:
            user_input = input("聊天：")
            user_input = user_input.strip()

            if "quit" in user_input or "Quit" in user_input:
                break

            # 首先根据当前的agent，判断每个事件的条件是不是吻合，如果吻合则加入搜索

            #


In [ ]:
class GameMaster:
    def __init__(self, agent = None):
        self.state = "Menu"
        if agent is None:
            self.agent = Agent()


    def run(self):
        while True:
            if self.state == "Menu":
                self.menu()
            elif self.state == "EventMaster":
                self.event_master()
                self.state = "Menu"
            elif self.state == "ChatMaster":
                self.chat_master()
            elif self.state == "Quit":
                break

    def menu(self):
        print("1. 随机一个事件")
        print("2. 自由聊天")
        print("或者输入Quit退出")
        choice = input("请选择一个选项: ")
        if choice == "1":
            self.state = "EventMaster"
        elif choice == "2":
            self.state = "ChatMaster"
        elif "quit" in choice or "Quit" in choice or "QUIT" in choice:
            self.state = "Quit"
        else:
            print("无效的选项，请重新选择")

    def event_master(self):
        event_master = EventMaster()
        event_master.run(self.agent)
        self.state = "Menu"

    def chat_master(self):
        chat_master = ChatMaster()
        chat_master.run(self.agent)
        self.state = "Menu"




In [ ]:
game_master = GameMaster()
game_master.run()

1. 随机一个事件
2. 自由聊天
请选择一个选项: 1
事件发生了！
按任意键继续...
1. 随机一个事件
2. 自由聊天
请选择一个选项: 2
聊天：

False
聊天：Quit
Quit
False
聊天：Quit
Quit
False
聊天：quit
quit
True
1. 随机一个事件
2. 自由聊天
请选择一个选项: quit
